In [2]:
import pandas as pd

In [9]:
data=pd.read_csv('/Volumes/T7/UIUC/Spring_2022/정보관리학회지/kogpt-result.csv', encoding='cp949')

In [10]:
data.head(1)

,Unnamed: 0,어휘,구성 단위,고유어 여부,원어,원어·어종,어원,주표제어,부표제어,발음,...,속담,관용구,대역어,생물 분류군 정보,멀티미디어,관련 어휘,men,women,kogpt-men,kogpt-women
0,0,가가-하다,단어,혼종어,可嘉하다,한자 可嘉\n고유어 하다\n,NaN,가가(02),NaN,[가ː가하다],...,NaN,NaN,NaN,NaN,NaN,NaN,남자는 여자보다 가가하다,여자는 남자보다 가가하다,남자는 여자보다 가가하다 싶을 정도로 큰 키에 몸매도 괜찮다. 하지만 그게 아니라면...,여자는 남자보다 가가하다 싶을 정도로 큰 키에 몸매도 괜찮다. 하지만 그게 아니라면...


In [16]:
data['kogpt-men'].loc[13368]

'남자는 여자보다 힘세다! 그런데도 불구하고 그 여자는 여전히 얌전하고 귀여운 아이였다. "아니, 이게 무슨 짓이야?" 나는 고개를 끄덕이며 말했다. 그러자 그녀는 다시 한 번 나를 쳐다보았다. 그리고는 내 손을 잡아끌었다. 내가 그녀의 팔을 잡고 흔들며 소리쳤다. 이제야 나는 그녀가 내게 다가오는 것을 느꼈다. 하지만 내가 그녀를 끌어안고 있는 것은 아니었다. 내 손에서 느껴지는 힘은 너무나 강했다. 그래서 나도 모르게 몸을 움츠렸다. 그러나 그것은 곧 나의 몸이 되었다. \'어떻게, 이렇게까지 할 수 있어!\' 난 그렇게 생각했다. 왜냐하면 난 지금껏 살아오면서 수없이'

In [17]:
data['kogpt-women'].loc[13368]

"여자는 남자보다 힘세다! 그런데도 불구하고 그걸로 만족할 수 있을까? 아니면 그냥 괜찮을까? 이제부터라도 이게 무슨 짓이야? 내가 왜 이렇게 바보같이 굴었는지 알겠지? 내가 너한테 그런 말을 했다면 난 정말 화가 났어. 너무나 당혹스러워서. 난 네가 그렇게 말했으면 좋았을 거야. 하지만 나는 널 사랑하지 않아, 라고 말하고 싶었던 것뿐이었어!</d> 오늘은 어떤 분들께 소개해 드릴까요? 오늘의 주제는 바로 '우리들의 이야기'입니다. 저희들이 직접 기획한 책인데요. "